## Bounce Rate: Case Study
- Bounce rate is a crucial metric in web analytics that measures the percentage of visitors who land on a website and leave without engaging in any further interactions. It indicates the effectiveness of a website in capturing and retaining the attention of its visitors.

- Given a dataset containing information about the bounce rate, session duration, and other features of website users, the goal is to analyze the bounce rate and its relationship with session duration.

Your task is to gain insights into user engagement and identify factors that contribute to a high or low bounce rate. Find answers to questions such as:

1. What is the overall bounce rate of the website? 
2. Are there any specific patterns or trends in the relationship between bounce rate and session duration? 
3. Can we identify segments of users with different bounce rate behaviours?

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = 'plotly_white'

In [2]:
data = pd.read_csv('bounce-rate.csv')
data.head()

,Client ID,Sessions,Avg. Session Duration,Bounce Rate
0,5.778476e+08,367,00:01:35,87.19%
1,1.583822e+09,260,00:01:04,29.62%
2,1.030699e+09,237,00:00:02,99.16%
3,1.025030e+09,226,00:02:22,25.66%
4,1.469968e+09,216,00:01:23,46.76%


- Before moving further let's check if there are any null vlaues in the data

In [3]:
data.isnull().sum()

Client ID                0
Sessions                 0
Avg. Session Duration    0
Bounce Rate              0
dtype: int64

- As we can see above there are no any null values in the data

- Now, let's have a look about each columns information

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Client ID              999 non-null    float64
 1   Sessions               999 non-null    int64  
 2   Avg. Session Duration  999 non-null    object 
 3   Bounce Rate            999 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 31.3+ KB


- The Avg. Session Duration and Bounce Rate columns are not numerical. We need to convert them into appropriate data types for this task. Here’s how we can prepare our data

In [5]:
# Convert the 'Avg. Session Duration' column
data['Avg. Session Duration'] = data['Avg. Session Duration'].str[1:]
data['Avg. Session Duration'] = pd.to_timedelta(data['Avg. Session Duration'])
data['Avg. Session Duration'] = data['Avg. Session Duration'] / pd.Timedelta(minutes=1)

# Convert the 'Bounce Rate' column
data['Bounce Rate'] = data['Bounce Rate'].str.rstrip('%%').astype('float')

data.head()

,Client ID,Sessions,Avg. Session Duration,Bounce Rate
0,5.778476e+08,367,1.583333,87.19
1,1.583822e+09,260,1.066667,29.62
2,1.030699e+09,237,0.033333,99.16
3,1.025030e+09,226,2.366667,25.66
4,1.469968e+09,216,1.383333,46.76


- In the above code, we removed the first character from each value in the “Avg. Session Duration” column, representing a unit of time. Then, we converted the values in the “Avg. Session Duration” column to a standardized time delta format, representing time durations. Then, we further converted the time delta values to minutes, providing the average session duration in a numeric format. Similarly, we removed the percentage sign from each value in the “Bounce Rate” column and converted them to float values, representing the bounce rate as decimals.

- Now, let's have a look at the descriptive statistics of the data

In [6]:
data.describe()

,Client ID,Sessions,Avg. Session Duration,Bounce Rate
count,9.990000e+02,999.000000,999.000000,999.000000
mean,1.036401e+09,32.259259,3.636520,65.307978
std,6.151503e+08,24.658588,4.040562,22.997270
min,1.849182e+05,17.000000,0.000000,4.880000
25%,4.801824e+08,21.000000,0.891667,47.370000
50%,1.029507e+09,25.000000,2.466667,66.670000
75%,1.587982e+09,35.000000,4.816667,85.190000
max,2.063338e+09,367.000000,30.666667,100.000000


- Now let's have a look at the correlation matrix before moving forward

In [7]:
# Exclude 'Client_Id' column from the dataset
data_without_id = data.drop('Client ID', axis=1)

# Calculate the correlation matrix
correaltion_matrix = data_without_id.corr()


,Sessions,Avg. Session Duration,Bounce Rate
0,367,1.583333,87.19
1,260,1.066667,29.62
2,237,0.033333,99.16
3,226,2.366667,25.66
4,216,1.383333,46.76
